In [ ]:
root = '/Users/admin/Documents/PhD/Code/perceptual-tuning-results/'

In [ ]:
# Uncomment for development/debugging
#%matplotlib inline


# Uncomment to plot finalized figures

import matplotlib as mpl
mpl.use("pgf")
pgf_with_custom_preamble = {
    "font.family": "serif", # use serif/main font for text elements
    "text.usetex": True,    # use inline math for ticks
    "pgf.rcfonts": False,   # don't setup fonts from rc parameters
    "pgf.preamble": [
         "\\usepackage{unicode-math}",  # unicode math setup
         "\\setmainfont{Doulos SIL}" # serif font via preamble
         ]
}
mpl.rcParams.update(pgf_with_custom_preamble)


from scone_phobia.utils.apply_analyses import fetch_data
import scone_phobia.metadata.add_metadata as add_metadata
import numpy as np
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as path
import matplotlib.patches as mpatches

### Utility functions

In [ ]:
# Functions to aggregate data to obtain an error rate for each available (contrast x test set) level in 4 conditions:
#  Jap. native/Am. Eng. native/Read native/Spontaneous native

def agg_res(df, dependent_var, df_is_resampled=False):
    """
    Aggregate results to contrast either model with different "training language"
    or different "training register", by setting the level of 'dependent_var'
    accordingly.
    """
    groupby_cols = [dependent_var, 'model type', 'contrast', 'test language', 'test register']
    if df_is_resampled:
        # ensure separate analysis for each resample
        groupby_cols = groupby_cols + ['batch ID', 'batch size', 'boot ID']
    agg_df = df.groupby(groupby_cols, as_index=False).mean()
    return agg_df


def split_levels(df, dependent_var, levels):
    # aggregate results and split them according to 'levels' of 'dependent_var'
    df_agg = agg_res(df, dependent_var)
    dfs = {}
    for level in levels:
        dfs[level] = df_agg[df_agg[dependent_var] == level]
    return dfs   
        

def merge_dfs(dfs, on, suffixes):
    # a limited extension of pandas.merge able to merge any number of dataframes
    # recursive (more readable than an iterative version with reduce)
    assert len(dfs) >= 2
    if len(dfs) == 2:
        res_df = pandas.merge(dfs[0], dfs[1], on=on, suffixes=suffixes)
    else:
        right_df = merge_dfs(dfs[1:], on, suffixes[1:])
        left_df = dfs[0]
        suffix = suffixes[0]
        for col in left_df:
            if not(col in on):
                # add suffix
                left_df[col + suffix] = left_df[col]
                del left_df[col]
        res_df = pandas.merge(left_df, right_df, on=on, suffixes=['', ''])
    return res_df


def groupby_testset_contrast(df, langs, regs, suffixes):
    merge_cols = ["contrast", "test language", "model type", 'test register']
    dfs_trainlang = split_levels(df, "training language", langs)
    dfs_trainreg = split_levels(df, "training register", regs)
    data = [dfs_trainlang[lang] for lang in langs] + [dfs_trainreg[reg] for reg in regs]
    suffixes = [suffixes[lang] for lang in langs] + [suffixes[reg] for reg in regs]
    res_df = merge_dfs(data, on=merge_cols, suffixes=suffixes)
    return res_df


### Load and prepare data

In [ ]:
# Loading minimal-pair errors

mp_folder = root + 'ABX/mp_scores'

# select relevant models
dpgmm = 'dpgmm_novtln_vad'
percTun_filt = lambda mp_fname: 'AMtri1_sat_small_LMtri1satsmall' in mp_fname\
                                  or 'mfcc_novtln' in mp_fname\
                                  or dpgmm in mp_fname

# get raw minimal-pair discriminatione errors (without any resampling for now)
dummy = lambda x: x
df = fetch_data(dummy, mp_folder, filt=percTun_filt, add_metadata=add_metadata.language_register)

In [ ]:
# Prepare data

langs = ['American English', 'Japanese']
regs = ['Read', 'Spontaneous']
suffixes = {'American English': "_AE",
            'Japanese': "_Jap",
            'Read': "_read",
            'Spontaneous': "_spont"}
data = groupby_testset_contrast(df, langs, regs, suffixes)

# filter out silences
def not_sil(contrast, silences):
    p1, p2 = contrast.split("-")
    return not(p1 in silences) and not(p2 in silences)

silences = ['NSN', 'SPN', 'SIL']
data = data[[not_sil(con, silences) for con in data['contrast']]]

### Plot figures

In [ ]:
# do that before...
#del data['training language_AE']
#del data['training language_Jap']
#del data['training register_spont']
#del data['training register_read']

d_l = data[['model type', 'contrast', 'test language', 'test register']].copy()
d_l['native advantage'] = [e_Jap-e_AE if test_lang == 'American English' else e_AE-e_Jap for test_lang, e_AE, e_Jap in zip(data['test language'], data['error_AE'], data['error_Jap'])]
d_l['native parameter'] = 'language'

d_r = data[['model type', 'contrast', 'test language', 'test register']].copy()
d_r['native advantage'] = [e_spont-e_read if test_reg == 'Read' else e_read-e_spont for test_reg, e_read, e_spont in zip(data['test register'], data['error_read'], data['error_spont'])]
d_r['native parameter'] = 'register'

data2 = pandas.concat([d_l, d_r])

In [ ]:
# 1142 read speech
# 1293 spontaneous speech
len(np.unique(data2['contrast']))


In [ ]:
# some quick stats
rr = data2[(data2['native parameter'] == 'register') & (data2['model type'] == 'dpgmm_novtln_vad')]


len(np.where(rr['native advantage'] >= 0)[0]) / float(len(rr))
# Read: 52.9772; 70.1401
# Spontaneous: 44.7796; 51.1988
# AE: 71.0315; 80.7608
# Jap.  69.4757; 82.9588

# register > 0
# 0.4862422997946612
# register < 0
# 0.3991786447638604
# register == 0
# 0.11457905544147844

#>=0 0.6008213552361397

# language > 0
# 0.7034907597535934
# language < 0
# 0.18275154004106775
# language == 0
# 0.1137577002053388

#>=0 0.8172484599589322

In [ ]:

g = sns.catplot(data=data2, x='test register', y='native advantage', hue='native parameter', row='test language', col='model type', kind='boxen',
                col_order=['dpgmm_novtln_vad', 'AMtri1_sat_small_LMtri1satsmall'], legend=False, aspect=.7)

g.set_xlabels('Test stimuli\'s register', fontsize=18)
for ax in g.axes.flatten():
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(23)


for i, axes_row in enumerate(g.axes):
    for j, ax in enumerate(axes_row):
        if i==1:
            ax.set_xticklabels(['Read', 'Spont.'], fontsize=23)
            ax.set_title('')
        if j==0:
            ax.set_ylabel('$\Delta$ABX error rate (in \%)', fontsize=23)
        if i==0:
            if j==0:
                ax.set_title('GMM\n(unsupervised)', fontsize=23)
            else:
                ax.set_title('HMM\n(supervised)', fontsize=23)
        ax.set_xlim([-.4, 1.4])
        ax.grid(axis='y')   
        ax.tick_params(axis='both', which='both', width=0, length=0)
        ax.set_axisbelow(True)
        ax.plot([-.4, 1.4], [0, 0], 'k-')
g.despine(left=True)
          
     
         
g.fig.tight_layout()
g.savefig(root + 'ABX/figures/native_advantage_supp.pdf')    

In [ ]:
# main figure

g = sns.catplot(data=data2, x='model type', y='native advantage', col='native parameter', kind='boxen', sharey=True,
                order=['dpgmm_novtln_vad'], hue='native parameter', dodge=False, aspect=.6)

labs = ['Native language\nadvantage', "`Native register\'\nadvantage"]
#g.set_yticklabels('\Delta', fontsize=20)  # '$\Delta$ABX error rate (in \%)'

for tick in g.axes[0,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(23)

for i, (ax, lab) in enumerate(zip(g.axes.flatten(), labs)):
    ax.xaxis.set_ticks([])
    ax.set_xlabel('')
    ax.set_title(lab, fontsize=23)
    ax.set_ylim([-20, 20])
    ax.set_xlim([-.4, .4])
    if i == 0:
        ax.set_ylabel('$\Delta$ABX error rate (in \%)', fontsize=23)
    ax.tick_params(axis='both', which='both', width=0, length=0)
    ax.set_axisbelow(True)
    ax.plot([-.4, .4], [0, 0], 'k-')
    ax.grid(axis='y')
g.despine(left=True, bottom=True)

g.fig.tight_layout()
g.savefig(root + 'ABX/figures/native_advantage_main.pdf')